<a href="https://colab.research.google.com/github/cdbm/F1DataAnalysis/blob/master/Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
results=pd.read_csv("https://raw.githubusercontent.com/cdbm/F1DataAnalysis/master/f1_data/results.csv")
lap_times=pd.read_csv("https://raw.githubusercontent.com/cdbm/F1DataAnalysis/master/f1_data/lap_times.csv")
races=pd.read_csv("https://raw.githubusercontent.com/cdbm/F1DataAnalysis/master/f1_data/races.csv")

In [3]:
results = results[results.position != '\\N']
results = results[results.milliseconds != '\\N']
results = results[results.fastestLapSpeed != '\\N']

In [4]:
races[races.circuitId == 6]

,raceId,year,round,circuitId,name,date,time,url
5,6,2009,6,6,Monaco Grand Prix,2009-05-24,12:00:00,http://en.wikipedia.org/wiki/2009_Monaco_Grand...
22,23,2008,6,6,Monaco Grand Prix,2008-05-25,12:00:00,http://en.wikipedia.org/wiki/2008_Monaco_Grand...
39,40,2007,5,6,Monaco Grand Prix,2007-05-27,12:00:00,http://en.wikipedia.org/wiki/2007_Monaco_Grand...
58,59,2006,7,6,Monaco Grand Prix,2006-05-28,14:00:00,http://en.wikipedia.org/wiki/2006_Monaco_Grand...
75,76,2005,6,6,Monaco Grand Prix,2005-05-22,14:00:00,http://en.wikipedia.org/wiki/2005_Monaco_Grand...
...,...,...,...,...,...,...,...,...
916,931,2015,6,6,Monaco Grand Prix,2015-05-24,12:00:00,http://en.wikipedia.org/wiki/2015_Monaco_Grand...
940,953,2016,6,6,Monaco Grand Prix,2016-05-29,12:00:00,https://en.wikipedia.org/wiki/2016_Monaco_Gran...
961,974,2017,6,6,Monaco Grand Prix,2017-05-28,12:00:00,https://en.wikipedia.org/wiki/2017_Monaco_Gran...
981,994,2018,6,6,Monaco Grand Prix,2018-05-27,13:10:00,https://en.wikipedia.org/wiki/2018_Monaco_Gran...


In [5]:
results["circuitId"] = results.raceId.map(races.set_index('raceId')['circuitId'])

In [6]:
results = results[results.circuitId == 6]

In [7]:
results.positionText.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15'], dtype=object)

In [8]:
results.positionOrder.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [9]:
races = results.raceId.unique()

In [10]:
results = results.drop('number', axis=1)
results = results.drop('statusId', axis=1)
results = results.drop('time', axis=1)
results = results.drop('fastestLapTime', axis=1)
results = results.drop('positionOrder', axis=1)
results = results.drop('positionText', axis=1)
results = results.drop('resultId', axis=1)

In [11]:
results

,raceId,driverId,constructorId,grid,position,points,laps,milliseconds,fastestLap,rank,fastestLapSpeed,circuitId
108,23,1,1,3,1,10.0,76,7242742,71,6,153.152,6
109,23,9,2,5,2,8.0,76,7245806,75,5,154.286,6
110,23,13,6,1,3,6.0,76,7247553,76,4,154.379,6
111,23,17,9,9,4,5.0,76,7262037,74,8,152.133,6
112,23,20,5,19,5,4.0,76,7267399,74,7,152.614,6
...,...,...,...,...,...,...,...,...,...,...,...,...
24306,1015,826,5,7,7,6.0,78,6263011,31,10,157.471,6
24307,1015,848,5,10,8,4.0,78,6263637,43,3,158.890,6
24308,1015,817,4,6,9,2.0,78,6269331,78,4,158.701,6
24309,1015,154,210,13,10,1.0,78,6269471,41,16,156.531,6


In [12]:
results.fastestLapSpeed = results.fastestLapSpeed.astype('float64')
results.milliseconds = results.milliseconds.astype('int64')
results.fastestLap = results.fastestLap.astype('int64')
results["rank"] = results["rank"].astype('int64')

In [13]:
results.dtypes

raceId               int64
driverId             int64
constructorId        int64
grid                 int64
position            object
points             float64
laps                 int64
milliseconds         int64
fastestLap           int64
rank                 int64
fastestLapSpeed    float64
circuitId            int64
dtype: object

In [14]:
results.position.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15'], dtype=object)

In [15]:
from sklearn.model_selection import train_test_split
x_results = results.drop('position', axis=1)
y_results = results['position']

In [ ]:
x_results

In [ ]:
y_results

In [18]:
Xtrain, Xtest, ytrain, ytest = train_test_split(x_results, y_results, random_state=1)

In [19]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=4)             
model.fit(Xtrain, ytrain)                  
prediction = model.predict(Xtest)   

In [20]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(ytest, prediction)
print(prediction.shape)
print(ytest.shape)
print('accuracy', accuracy)

(36,)
(36,)
accuracy 0.05555555555555555


In [88]:
qualifying=pd.read_csv("https://raw.githubusercontent.com/cdbm/F1DataAnalysis/master/f1_data/qualifying.csv")

In [96]:
qualifying = qualifying[qualifying.q1 != '\\N']
qualifying = qualifying[qualifying.q2 != '\\N']
qualifying = qualifying[qualifying.q3 != '\\N']
qualifying.dtypes

qualifyId          int64
raceId             int64
driverId           int64
constructorId      int64
number             int64
position           int64
q1               float64
q2               float64
q3               float64
dtype: object

In [95]:
from datetime import datetime

qualifying.q1 = [(pd.to_datetime(qtime, format='%M:%S.%f') - datetime(1970,1,1)).total_seconds() for qtime in qualifying.q1]

qualifying.q2 = [(pd.to_datetime(qtime, format='%M:%S.%f') - datetime(1970,1,1)).total_seconds() for qtime in qualifying.q2]
qualifying.q3 = [(pd.to_datetime(qtime, format='%M:%S.%f') - datetime(1970,1,1)).total_seconds() for qtime in qualifying.q3]


In [97]:
qualifying

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,-2.208989e+09,-2.208989e+09,-2.208989e+09
1,2,18,9,2,4,2,-2.208989e+09,-2.208989e+09,-2.208989e+09
2,3,18,5,1,23,3,-2.208989e+09,-2.208989e+09,-2.208989e+09
3,4,18,13,6,2,4,-2.208989e+09,-2.208989e+09,-2.208989e+09
4,5,18,2,2,3,5,-2.208989e+09,-2.208989e+09,-2.208989e+09
...,...,...,...,...,...,...,...,...,...
8459,8483,1036,848,9,23,6,-2.208989e+09,-2.208989e+09,-2.208989e+09
8460,8484,1036,832,1,55,7,-2.208989e+09,-2.208989e+09,-2.208989e+09
8461,8485,1036,846,1,4,8,-2.208989e+09,-2.208989e+09,-2.208989e+09
8462,8486,1036,844,6,16,9,-2.208989e+09,-2.208989e+09,-2.208989e+09


In [98]:
qualifying.dtypes

qualifyId          int64
raceId             int64
driverId           int64
constructorId      int64
number             int64
position           int64
q1               float64
q2               float64
q3               float64
dtype: object

In [99]:
from sklearn.model_selection import train_test_split
x = qualifying.drop('position', axis=1)
y = qualifying['position']
Xtrain, Xtest, ytrain, ytest = train_test_split(x, y, random_state=1)

In [100]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=4)             
model.fit(Xtrain, ytrain)                  
prediction = model.predict(Xtest)  

ValueError: ignored

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(ytest, prediction)
print(prediction.shape)
print(ytest.shape)
print('accuracy', accuracy)